In [23]:
import plotly.express as plt 
import pandas as pd
import os
import numpy as np
import sys 
sys.path.append('./modules/')
from mva import *

In [24]:
matrix = np.array([
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0.004,0.006,0.9,0.065,0.025],
    [0,0,1,0,0],
    [0,0,1,0,0]],np.dtype('d'))

In [25]:
mva = MVA(matrix,[5000,210,2.7,40,180],[StationType.Delay,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent],30)

def mvaToDataframe(matrix: np.ndarray)-> pd.DataFrame:
    vals = pd.DataFrame()
    vals["DELAY"] = matrix[0]
    vals["SWAP_IN"] = matrix[1]
    vals["CPU"] = matrix[2]
    vals["IO1"] = matrix[3]
    vals["IO2"] = matrix[4]
    vals["N"] = [i for i in range(0,len(matrix[0]))]
    return vals

mva()

In [26]:
#utilization = pd.read_csv("../../build/Scheduler/utilizations.csv",sep=";")
#throughputs = pd.read_csv("../../build/Scheduler/throughputs.csv", sep=";")
#meanWaits = pd.read_csv("../../build/Scheduler/meanWaits.csv",sep=";")
#meanClients = pd.read_csv("../../build/Scheduler/meanClients.csv", sep=";")

utilization = mvaToDataframe(mva.utilizations)
throughputs = mvaToDataframe(mva.throughputs)
meanWaits = mvaToDataframe(mva.meanwaits)
meanClients = mvaToDataframe(mva.meanclients)

visits = {}
visits["CPU"] = 250
visits["SWAP_IN"] = 2.5
visits["IO1"] = 16.25
visits["IO2"] = 6.25 

In [27]:


responseTimes = {}
responseTimes["CPU"] = meanWaits["CPU"]*visits["CPU"]
responseTimes["SWAP_IN"] = meanWaits["SWAP_IN"]*visits["SWAP_IN"]
responseTimes["IO1"] = meanWaits["IO1"]*visits["IO1"]
responseTimes["IO2"] =meanWaits["IO2"]*visits["IO2"]
responseTimes["N"] = meanWaits["N"]
print(responseTimes)


{'CPU': 0     0.000000e+00
1     6.750000e+02
2     2.497500e+03
3     7.418250e+03
4     2.070428e+04
5     5.657654e+04
6     1.534317e+05
7     4.149405e+05
8     1.121014e+06
9     3.027414e+06
10    8.174692e+06
11    2.207234e+07
12    5.959600e+07
13    1.609099e+08
14    4.344574e+08
15    1.173036e+09
16    3.167197e+09
17    8.551432e+09
18    2.308887e+10
19    6.233994e+10
20    1.683178e+11
21    4.544582e+11
22    1.227037e+12
23    3.313000e+12
24    8.945100e+12
25    2.415177e+13
26    6.520978e+13
27    1.760664e+14
28    4.753793e+14
29    1.283524e+15
Name: CPU, dtype: float64, 'SWAP_IN': 0     0.000000e+00
1     5.250000e+02
2     1.107750e+05
3     2.326328e+07
4     4.885288e+09
5     1.025911e+12
6     2.154412e+14
7     4.524265e+16
8     9.500957e+18
9     1.995201e+21
10    4.189922e+23
11    8.798837e+25
12    1.847756e+28
13    3.880287e+30
14    8.148603e+32
15    1.711207e+35
16    3.593534e+37
17    7.546421e+39
18    1.584748e+42
19    3.327972e+44
20  

In [28]:
def produce_graph_and_save(dataframe: pd.DataFrame, name:str, yaxis:str, log:bool = False):
    fig = plt.line(dataframe,y=["CPU","SWAP_IN","IO1","IO2"], x="N",title=name,log_y=log)
    fig.update_yaxes(title_text=yaxis)
    data = fig.to_image("png")
    f = open("./Images/{}.png".format(name),"+wb")
    f.write(data)
    f.close()
    fig.show()
pass

In [29]:
produce_graph_and_save(utilization,"Utilizations","B/T")

In [30]:
produce_graph_and_save(meanWaits,"meanWaits","ms",True)

In [31]:
produce_graph_and_save(meanClients,"meanClients","n")

In [32]:
produce_graph_and_save(throughputs,"throughputs","C/T")

In [33]:
produce_graph_and_save(responseTimes,"Response_Times","ms",True)

In [34]:
newmat = matrix.astype(np.double)
old = newmat.astype(np.double)
for i in range(50000):
    old = newmat.astype(np.double)
    newmat = old.astype(np.double) @ old.astype(np.double)
    newmat = newmat.round(10) #needed because overflow may occur
    pass
print(newmat)
pi = np.array([0]*len(matrix))
pi[0] = 1
pi = pi@newmat
pi.sum()

[[0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]
 [0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]
 [0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]
 [0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]
 [0.00362319 0.00905797 0.9057971  0.05887681 0.02264493]]


1.0

In [35]:
app = [0]*len(pi)
for i in range(len(pi)):
    app[i] = pi[i]*newmat[i][i]
    pass
app

[1.312749418189456e-05,
 8.2046838636841e-05,
 0.8204683890858013,
 0.003466478944181895,
 0.0005127927414802562]